In [2]:
import arcpy

# Set the workspace
arcpy.env.workspace = r"D:\hotspots\fencing_crossings\fencing_crossings.gdb"
arcpy.env.overwriteOutput = True

# Input feature classes
points_fc = "water_bridges"
lines_fc = "wwc_58R"

# Create temporary feature classes
split_lines_fc = "in_memory/split_lines"
search_segments_fc = "in_memory/search_segments"

# Split the lines at each point
arcpy.management.SplitLineAtPoint(lines_fc, points_fc, split_lines_fc, "0.01 Meters")

# Function to search along the roadways
def search_along_roadways(points_fc, split_lines_fc, search_distance):
    arcpy.management.CreateFeatureclass("in_memory", "search_segments", "POLYLINE", spatial_reference=split_lines_fc)
    with arcpy.da.SearchCursor(points_fc, ["SHAPE@"]) as point_cursor:
        with arcpy.da.InsertCursor("in_memory/search_segments", ["SHAPE@"]) as segment_cursor:
            for point in point_cursor:
                point_geom = point[0]
                with arcpy.da.SearchCursor(split_lines_fc, ["SHAPE@", "OID@"]) as line_cursor:
                    for line in line_cursor:
                        line_geom = line[0]
                        if line_geom.distanceTo(point_geom) <= search_distance:
                            segment_cursor.insertRow([line_geom])
    return "in_memory/search_segments"

# Define the search distance (2 miles)
search_distance = 2 * 1609.34  # 2 miles in meters

# Search along the roadways and create the search segments
search_segments = search_along_roadways(points_fc, split_lines_fc, search_distance)

# Perform spatial join based on the value from the line field
spatial_join_fc = "spatial_join"
arcpy.analysis.SpatialJoin(search_segments, lines_fc, spatial_join_fc, "JOIN_ONE_TO_MANY", "KEEP_COMMON", 
                           match_option="INTERSECT")

# Optional: Export the result to a feature class for further analysis
output_fc = "C:/path/to/your/output/spatial_join_result"
arcpy.management.CopyFeatures(spatial_join_fc, output_fc)

# Verify the results
print(f"Spatial join results saved to: {output_fc}")


ExecuteError: ERROR 000622: Failed to execute (Create Feature Class). Parameters are not valid.
ERROR 000628: Cannot set input into parameter spatial_reference.
